# Test GPU

In [ ]:
import torch
torch.cuda.current_device()
torch.cuda.get_device_name(0)

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
Folder => "Alzheimer Competition"
https://drive.google.com/drive/folders/1-NIobsSrpU5JyUfu0bqbuWZU32GVZ7-m?usp=sharing

Folder => "SayeedColab"
https://drive.google.com/drive/folders/1wnL3y-kltnGwsV9dhha6BbOI7UwF0YJW?usp=sharing

"""

# Importing Necessary Libraries

In [ ]:
import shutil
import os
import h5py
import csv

# importing the libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import time
import glob
import gc

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import torch.utils.data

%matplotlib inline


**TIP:** This training could take several hours depending on how many iterations you chose in the .cfg file. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit **Enter**
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```

# Copy Data

In [ ]:
import shutil
import os
import glob

shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/resnet.py", "resnet.py")
shutil.copyfile("/content/drive/My Drive/Alzheimer Competition/submission_format.csv", "submission_format.csv")


submission_format_csv = "submission_format.csv"

print("Done")

### RESNET101 Data Import

In [ ]:
print("This section is going to take about 20 minutes time")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_1.zip";
print("partition 1 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_2.zip";
print("partition 2 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_3.zip";
print("partition 3 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_4.zip";
print("partition 4 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_5.zip";
print("partition 5 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_6.zip";
print("partition 6 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_7.zip";
print("partition 7 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_8.zip";
print("partition 8 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_9.zip";
print("partition 9 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_10.zip";
print("partition 10 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_11.zip";
print("partition 11 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_12.zip";
print("partition 12 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_13.zip";
print("partition 13 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_14.zip";
print("partition 14 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/test_15.zip";
print("partition 15 imported")


path = "./test/"
files = [f for f in glob.glob("test/" + "*" + ".pt", recursive=True)]
print("Total: " + str(len(files)) + " should be 14160")

# Mandatory Section

In [ ]:
def string2torch(test_y):
  df = pd.DataFrame(test_y, columns = ['Fname'])
  y = (df['Fname'].values)

  for i,filename in enumerate(y):
    seq_name = filename.split(".mp4")[0]
    y[i] = seq_name

  processed = np.array(y)
  processed = processed.astype(np.int)
  processed = torch.from_numpy(processed)

  return processed

In [ ]:
def make_submission_file(filenames, y_pred):
  submit = []
  filenames = filenames.astype(int)
  for i in filenames:
    submit += [str(i)+'.mp4']

  submission_dict = {"filename": submit, "stalled": y_pred.astype(int)}
  submission_csv = pd.DataFrame(submission_dict)
  submission_csv.to_csv("submission3D.csv", index=False)

In [ ]:
with open(submission_format_csv, mode='r') as infile:
    reader = csv.reader(infile)
    test_list_csv = {rows[0]: rows[1] for rows in reader}
    infile.close()


files = list(test_list_csv.keys())
files.pop(0)
test_len = len(files)
print("Total data: " + str(test_len))

# RESNET101 Inference

### Model

In [ ]:
depth, height, width = 32, 64, 64   # dimension for converting point cloud to voxels

shutil.copyfile("/content/drive/My Drive/Alzheimer Competition/lr1e-4_dec8e-4_90.484_mcc_0.766.pth", "weight_3D.pth")
checkpoint_model = "weight_3D.pth"

# fold3_resnet101_aug16_32_64_64_acc_85.476_mcc_0.641.pth
# lr1e-4_dec1e-3_acc_89.649_mcc_0.744.pth
# lr1e-4_dec8e-4_90.484_mcc_0.766.pth

In [ ]:
import resnet
model = resnet.resnet101(
                num_classes=2,
                shortcut_type='B',
                sample_size=64,
                sample_duration=32)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.load_state_dict(torch.load(checkpoint_model))

### Resnet101 Submission

In [ ]:
y_pred = []
filenames = np.array([])


big_batch_size = 1024   # number of test data loaded at a time


batch_size = 128


for big_batch_no in range(math.ceil(test_len/big_batch_size)):

  this_batch_len = big_batch_size
  if (test_len - big_batch_size * big_batch_no) < big_batch_size:
    this_batch_len = test_len - big_batch_size * big_batch_no


  test_x = np.zeros((this_batch_len, 3, depth, height, width), dtype=np.float32)
  test_yy = np.zeros(this_batch_len)


  # Load one big batch
  for i in tqdm(range(this_batch_len)):

    original_idx = i + big_batch_size * big_batch_no

    f = files[original_idx]
    original_name = f.replace(".mp4", "")

    test_x[i, :, :, :, :] = torch.load(path + original_name + ".pt")
    test_yy[i] = int(original_name)

    # print(original_idx, original_name, f, h5_filename)

  
  # Test one big batch
  test_y = torch.from_numpy(test_yy).int()
  ## change the data types from here
  test_x = torch.from_numpy(test_x).float()
  #test_y = torch.from_numpy(test_y)
  
  test = torch.utils.data.TensorDataset(test_x, test_y)
  test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False, num_workers=4)

  model.eval()
  for i,(images,labels) in tqdm(enumerate(test_loader)):
      #print(labels)
      filenames = np.append(filenames, labels)
      
      images = images.view(-1,3,depth,height,width)
      test = Variable(images.to(device), requires_grad=False)
      labels = Variable(labels.to(device), requires_grad=False)

      with torch.no_grad():
        # Forward propagation
        outputs = model(test)

        # Get predictions from the maximum value
        predicted = torch.max(outputs.data, 1)[1]
        #print(f"prediction size are {predicted.shape}")
        y_pred = np.append(y_pred, predicted.cpu().numpy())

#print(filenames)

make_submission_file(filenames, y_pred)

print("Done")